In [1]:
from distributed import Client as DaskClient
import sqlalchemy as sa

from classify.features import *

In [2]:
DaskClient()

distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-worker-space/worker-9b9vdo07', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-worker-space/worker-3e25vjrg', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-worker-space/worker-1rmczr44', purging
distributed.diskutils - INFO - Found stale lock file and directory '/Users/isaak/repos/LAJC-expungement/dask-worker-space/worker-vylakfqf', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 8.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53153,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 8.00 GiB
Comm: tcp://127.0.0.1:53166,Total threads: 1
Dashboard: http://127.0.0.1:53169/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:53159,


In [3]:
pd.set_option('max_columns', None)

In [4]:
config = ExpungeConfig.from_yaml('classify/expunge_config.yaml')

config.years_until_next_conviction_short

numpy.timedelta64(7,'Y')

In [5]:
query = (
    sa.sql.select(expunge_model)
        # Where clause just for testing, comment out for full run
        # .where(
        #     sa.or_(
        #         # expunge_model.c.person_id == 127051000000102, 
        #         # expunge_model.c.person_id == 224010000000817,
        #         # expunge_model.c.person_id == 1000000000362

        #         expunge_model.c.person_id == 1021000000606,
        #         expunge_model.c.person_id == 1070000000994
        #     )
        # )
        .order_by(
            expunge_model.c.person_id,
            expunge_model.c.HearingDate
        )
)

In [6]:
ddf = fetch_expunge_data(custom_query=query)

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips
person_id,,,,,,,,,
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195


In [7]:
ddf = clean_data(ddf)

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips
person_id,,,,,,,,,
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195


In [14]:
ddf = (
    ddf.pipe(build_disposition)
       .pipe(build_chargetype)
       .pipe(build_codesection, config)
       .pipe(build_convictions)
       .pipe(build_date_features)
       .pipe(build_timedelta_features, config)
       .pipe(build_class_features)
)

ddf.head()

,HearingDate,CodeSection,ChargeType,Class,DispositionCode,Plea,Race,Sex,fips,disposition,chargetype,codesection,convictions,last_hearing_date,last_felony_conviction_date,next_conviction_date,last_hearing_delta,last_felony_conviction_delta,next_conviction_delta,from_present_delta,arrest_disqualifier,felony_conviction_disqualifier,next_conviction_disqualifier_short,next_conviction_disqualifier_long,from_present_disqualifier_short,from_present_disqualifier_long,is_class_1_or_2,is_class_3_or_4,class1_2,class3_4
person_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,NaT,NaT,2010-03-17,NaT,NaT,614 days,4556 days,False,False,True,True,False,False,False,False,False,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,2008-07-11,NaT,2010-03-17,0 days,NaT,614 days,4556 days,True,False,True,True,False,False,False,False,False,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,2008-07-11,NaT,2010-03-17,0 days,NaT,614 days,4556 days,True,False,True,True,False,False,False,False,False,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,2008-07-11,NaT,2010-03-17,0 days,NaT,614 days,4556 days,True,False,True,True,False,False,False,False,False,False
1001000000103,2008-07-11,18.2-172,Felony,5,Nolle Prosequi,NA,Black,Female,195,Dismissed,Felony,covered elsewhere,True,2008-07-11,NaT,2010-03-17,0 days,NaT,614 days,4556 days,True,False,True,True,False,False,False,False,False,False


In [13]:
ddf = (
    ddf.pipe(build_disposition)
       .pipe(build_chargetype)
       .pipe(build_codesection, config)
       .pipe(build_convictions)
       .pipe(build_date_features)
       .pipe(build_timedelta_features, config)
       .pipe(build_class_features)
)

ddf.head()